<a href="https://colab.research.google.com/github/IvanDePivan/2AMM30-groep-2/blob/main/Flairmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining Group 2
This notebook was developed for use on Google Colab. To run the notebook make sure the files from the data folder are uploaded.




## Prerequisites

In [1]:
from google.colab import output
!pip uninstall -y torch
!pip install flair

import os
import random
import json
import re
import regex
import difflib
import srsly
import torch
from typing import List
from pathlib import Path

from flair.trainers import ModelTrainer
from flair.data import Corpus, Label, Relation, Sentence
from flair.datasets import ColumnCorpus, DataLoader
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TokenEmbeddings, TransformerWordEmbeddings, TransformerDocumentEmbeddings
from flair.models import SequenceTagger, RelationExtractor, RelationClassifier

output.clear()

In [ ]:
# Sanity check on the gpu; some issues with flair not recognizing one of the GPUs from Colab
torch.cuda.is_available()

True

### Reformatting Label Studio output
We wrote this method for converting the output from Label Studio to the conll format, because the standard format had issues (e.g. No relation data was exported, tokenization had flaws, output used different delimiters)

In [ ]:
def write_conll(filename, text):
  output_file = Path(filename)
  output_file.parent.mkdir(exist_ok=True, parents=True)
  with open(output_file, "w") as file1:
    file1.write("\n\n".join(text))

In [ ]:
# Function for getting each token with their correct label and id (ids are used for relations)
def get_labels(file_item, key_field):
  rows = []
  position = 0
  next_label = 0
  # Sort the labels on start?
  labels = [item for item in file_item['annotations'][0]['result'] if item['type'] == 'labels']
  labels.sort(key=lambda x: x['value']['start'])
  item_count = 1
  b_count = 0
  ids = {} # count -> id
  tokens = [item for item in re.split(r'(\s+)|([\(,\.\)\\;:$*&"\-–])|(\')|(\'\w+)', file_item['data'][key_field]) if item]
  for i, item in enumerate(tokens):
    if re.search(r'\s+', item):
      position += len(item)
      continue
    start = position
    end = position + len(item)
    row = f'{item_count}\t{item}'
    label = '\tO'

    if len(labels) > next_label:
      if start >= labels[next_label]['value']['start']:
          # the current item has a label, and might have a relation
          id = labels[next_label]["id"]
          ids[item_count] = id
          if end > labels[next_label]['value']['end']:
            difference = end - labels[next_label]['value']['end']
            end -= difference
            tokens.insert(i+1, item[-difference:])
            item = item[:-difference]
          if start == labels[next_label]['value']['start']:
            # Current Item is start of a label
            b_count += 1
            label = '\tB-' + labels[next_label]["value"]["labels"][0]

          else:
            # Current item is inside a label
            if '\tB-' + labels[next_label]["value"]["labels"][0] in rows[-1] or '\tI-' + labels[next_label]["value"]["labels"][0] in rows[-1]:
              label = '\tI-' + labels[next_label]["value"]["labels"][0]
            else:
              b_count += 1
              label = '\tB-' + labels[next_label]["value"]["labels"][0]

      if end >= labels[next_label]['value']['end']:
        next_label += 1


    row = row + label
    position += len(item)
    item_count += 1
    rows.append(row)
  # Sanity check on the labels, gives a warning if labels are disappearing.
  if len(labels) != b_count:
    print(f'labels: {len(labels)}, B-tags: {b_count}')
    print(labels)
    print(ids)
    print(f'missing label: {[item for item in labels if item["id"] not in ids.values()]}')
    print(rows)
  return rows, ids


In [ ]:
# This creates the relation format flair expects for each document
def get_relations(file_item, ids):
  keys = ids.keys()
  relations = [item for item in file_item['annotations'][0]['result'] if item['type'] == 'relation']
  bio_relations = []
  found_relations = 0
  for relation in relations:
    from_ids = []
    to_ids = []
    rel_type = relation['labels'][0]
    for key in keys:
      if ids[key] == relation['from_id']:
        from_ids.append(key)
      if ids[key] == relation['to_id']:
        to_ids.append(key)
    from_ids = [int(i) for i in from_ids]
    from_ids.sort()
    to_ids = [int(i) for i in to_ids]
    to_ids.sort()
    if len(from_ids) > 0 and len(to_ids) > 0:
      found_relations += 1
      bio_relations.append(f'{from_ids[0]};{from_ids[-1]};{to_ids[0]};{to_ids[-1]};{rel_type}')
  if found_relations != len(relations):
    print(f'found relations: {found_relations}, actual relations: {len(relations)}')
    print(relations)
    print(bio_relations)
    print(keys)
    print(ids)
  return bio_relations

In [ ]:
# creates the connl file using the expanded json file as input. This is reused for component 2
def make_conll(file, key_field='text', file_name='test_set'):
  conllu_file = []
  for item in file:
    doc = ''
    rows, ids = get_labels(item, key_field)
    relations = get_relations(item, ids)
    if relations and len(relations) > 0:
      doc += '# relations = ' + ('|'.join(relations))
    doc += '\n' + ('\n'.join(rows))
    conllu_file.append(doc)

  print(len(conllu_file))
  write_conll(f'{file_name}.conll', conllu_file)

## Component 1
First, the json files are converted to the appropriate format (conll).

In [ ]:
# Make a conll file from the test set json
with open('comp1_test_set.json') as f:
  file = json.load(f)
  make_conll(file, file_name='data/comp1_test_set')

# Make a conll file from the train set json
with open('comp1_train_set.json') as f:
  file = json.load(f)
  make_conll(file, file_name='data/comp1_train_set')


40
95


In [ ]:
# The training conll file is split up for train and dev data
with open('data/comp1_train_set.conll') as f:
  data = f.read()
  splits = data.split(2*os.linesep)
  print(len(splits))
  splits = sorted(splits, key=lambda x: random.random())

  # For component 1, uses the last 25 for dev
  train = splits[:-25]
  dev = splits[-25:]

  write_conll('data/comp1/train.conll', train)
  write_conll('data/comp1/dev.conll', dev)

with open('data/comp1_test_set.conll') as f:
  data = f.read()
  splits = data.split(2*os.linesep)
  write_conll('data/comp1/test.conll', splits)

95


A corpus is created from the separate conll files. We specify which column in the conll files contains the NER labels.

In [ ]:
# The corpus converts the BIO formatted tokens into a structure Flair uses for training
# Here we can simply specify the folder where the files are and which columns they have
columns = {1: 'text', 2: 'ner'}
data_folder = 'data/comp1/'

corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll',
                              dev_file='dev.conll')

# Here we check if the corpus has the expected size, meaning importing likely went well
print(f'Corpus size: {len(corpus.train)}')

2023-11-10 09:29:09,100 Reading data from data/comp1
2023-11-10 09:29:09,105 Train: data/comp1/train.conll
2023-11-10 09:29:09,106 Dev: data/comp1/dev.conll
2023-11-10 09:29:09,112 Test: data/comp1/test.conll
Corpus size: 70


In [ ]:
# this dictionary let's the model know what labels exist in the set.
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

2023-11-10 09:29:09,885 Computing label dictionary. Progress:


70it [00:00, 6861.61it/s]

2023-11-10 09:29:09,922 Dictionary created for label 'ner' with 6 values: Winner (seen 605 times), Date (seen 422 times), Nationality (seen 331 times), Prizetype (seen 69 times), Reason (seen 44 times)


In [ ]:
# The embedding layer uses a pretrained model, distilBERT, cased in this case
# The layers="-1" specifies which layer to get the embeddings from. We use the final layer
# fine_tuning is on, as we only use a linear layer on top
# Subtoken pooling is set to first, meaning only the first subword is considered for the embeddings
# However, context is also considered
embeddings = TransformerWordEmbeddings(
    model='distilbert-base-cased',
    layers="-1",
    subtoken_pooling='first',
    fine_tune=True,
    use_context=True,
    respect_document_boundaries=False,
)

# This tagger put on top of the embeddings and does the classification
# Here we specify BIO vs BIOES and supply the label (tag) dictionary, and have additional choices in model config
tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,
        embeddings=embeddings,
        tag_dictionary=tag_dictionary,
        tag_type=tag_type,
        use_crf=False,
        use_rnn=False,
        reproject_embeddings=False,
        tag_format='BIO'
    )

2023-11-10 09:29:16,751 SequenceTagger predicts: Dictionary with 11 tags: O, B-Winner, I-Winner, B-Date, I-Date, B-Nationality, I-Nationality, B-Prizetype, I-Prizetype, B-Reason, I-Reason


In [ ]:
# The model trainer in flair makes finetuning very easy
trainer : ModelTrainer = ModelTrainer(tagger, corpus)

# Note this is set for a decent graphics card (like a v100 on Colab)
trainer.fine_tune(
        'data/comp1/ner',
        learning_rate=5e-05,
        mini_batch_size=16,
        max_epochs=100,
        embeddings_storage_mode='gpu', # cpu, gpu
        weight_decay=0.0,
    )


2023-11-10 09:29:16,793 ----------------------------------------------------------------------------------------------------
2023-11-10 09:29:16,804 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(28997, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_fea

100%|██████████| 2/2 [00:03<00:00,  1.91s/it]

2023-11-10 09:29:27,995 Evaluating as a multi-label problem: False
2023-11-10 09:29:28,116 DEV : loss 2.1376683712005615 - f1-score (micro avg)  0.0065
2023-11-10 09:29:28,147 ----------------------------------------------------------------------------------------------------


2023-11-10 09:29:28,718 epoch 2 - iter 1/5 - loss 2.24434881 - time (sec): 0.56 - samples/sec: 7011.70 - lr: 0.000006
2023-11-10 09:29:29,313 epoch 2 - iter 2/5 - loss 2.26072765 - time (sec): 1.16 - samples/sec: 7312.26 - lr: 0.000007
2023-11-10 09:29:29,987 epoch 2 - iter 3/5 - loss 2.22494892 - time (sec): 1.83 - samples/sec: 7208.25 - lr: 0.000008
2023-11-10 09:29:30,603 epoch 2 - iter 4/5 - loss 2.19728792 - time (sec): 2.45 - samples/sec: 7389.10 - lr: 0.000008
2023-11-10 09:29:30,904 epoch 2 - iter 5/5 - loss 2.17338455 - time (sec): 2.75 - samples/sec: 7193.35 - lr: 0.000009
2023-11-10 09:29:30,909 ----------------------------------------------------------------------------------------------------
2023-11-10 09:29:30,913 EPOCH 2 done: loss 2.1734 - lr 0.000009


100%|██████████| 2/2 [00:03<00:00,  1.97s/it]

...

2023-11-10 09:40:08,748 Evaluating as a multi-label problem: False
2023-11-10 09:40:08,772 DEV : loss 0.3713493049144745 - f1-score (micro avg)  0.8282


2023-11-10 09:40:09,426 ----------------------------------------------------------------------------------------------------
2023-11-10 09:40:09,430 Testing using last state of model ...


100%|██████████| 3/3 [00:01<00:00,  1.95it/s]


2023-11-10 09:40:10,984 Evaluating as a multi-label problem: False
2023-11-10 09:40:11,016 0.7869	0.8692	0.826	0.7042
2023-11-10 09:40:11,017 
Results:
- F-score (micro) 0.826
- F-score (macro) 0.7454
- Accuracy 0.7042

By class:
              precision    recall  f1-score   support

      Winner     0.7172    0.8087    0.7602       345
        Date     0.9614    0.9881    0.9746       252
 Nationality     0.8201    0.8807    0.8493       176
   Prizetype     0.7347    0.8780    0.8000        41
      Reason     0.2791    0.4444    0.3429        27

   micro avg     0.7869    0.8692    0.8260       841
   macro avg     0.7025    0.8000    0.7454       841
weighted avg     0.7987    0.8692    0.8316       841

2023-11-10 09:40:11,026 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8259887005649718,
 'dev_score_history': [0.006514657980456026,
  0.00198609731876862,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.1009771986970684,
  0.486039296794209,
  0.5283381364073005,
  0.6701298701298701,
  0.745067087608524,
  0.7431972789115646,
  0.7036688617121355,
  0.7839357429718877,
  0.7677806341045417,
  0.785063752276867,
  0.7657316148597422,
  0.76171875,
  0.7946074544012688,
  0.7862318840579711,
  0.8137009189640769,
  0.8099742046431642,
  0.8216398985629755,
  0.8148148148148148,
  0.8064777327935222,
  0.8257510729613733,
  0.8042588042588042,
  0.8181818181818181,
  0.8160535117056856,
  0.8151986183074266,
  0.8195364238410596,
  0.8120805369127518,
  0.8206839032527106,
  0.8230313293818797,
  0.8130360205831904,
  0.840677966101695,
  0.8309012875536481,
  0.8301255230125523,
  0.8053797468354431,
  0.8214285714285714,
  0.8246913580246914,
  0.831918505942275,
  0.8268733850129198,
  0.8162926018287614,
  0.8203842940685045,
  0.8032128514056

In [ ]:
# Sanity check on the model output

# load the trained model
model = SequenceTagger.load("data/comp1/ner/final-model.pt")

# create example sentence
sentence = Sentence('Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer. ')

# predict the tags
model.predict(sentence)
print(sentence.to_tagged_string())

2023-11-10 09:40:12,646 SequenceTagger predicts: Dictionary with 11 tags: O, B-Winner, I-Winner, B-Date, I-Date, B-Nationality, I-Nationality, B-Prizetype, I-Prizetype, B-Reason, I-Reason
Sentence[79]: "Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer." → ["Barack Hussein Obama II"/Winner, "August 4, 1961"/Date, "American"/Nationality, "the United States"/Nationality, "he"/Winner, "Obama"/Winner, "U.S."/Nationality]


In [ ]:
# The corpus from before also contains the relations, for example:
# relations = 1;5;7;9;born_on|1;5;11;13;died_on|1;5;17;17;has_nationality
# using indices per word to denote spans for each relation.
# Theoretically, this would allow for subspan relations, but this is not utilised here as we picked BIO, which has no nesting
relation_dictionary = corpus.make_label_dictionary("relation")

# The relation classifier works with document embeddings instead of word embeddings.
# This essentially encodes the whole document at once, but accounts for maximum sequence length
doc_embeddings = TransformerDocumentEmbeddings(model="distilbert-base-uncased", layers="-1", fine_tune=True)

# Flair has 2 RE models (RelationExtractor and RelationClassifier)
# The one we use (RelationClassifier) has significantly more performance
relation_model: RelationClassifier = RelationClassifier(
    embeddings =doc_embeddings,
    label_dictionary = relation_dictionary,
    label_type="relation",
    entity_label_types="ner",
    entity_pair_labels={  # Define valid entity pair combinations, used as relation candidates
        ("Winner", "Nationality"),
        ("Winner", "Date"),
        ("Winner", "Prizetype"),
        ("Winner", "Reason"),
    },
    zero_tag_value='',
    allow_unk_tag=False,
)

In [ ]:
trainer: ModelTrainer = ModelTrainer(model=relation_model, corpus=relation_model.transform_corpus(corpus))

In [ ]:
trainer.fine_tune(
    'data/comp1/relations/',
    max_epochs=20,
    learning_rate=4e-5,
    mini_batch_size=8,
    main_evaluation_metric=("macro avg", "f1-score"),
    shuffle=True,
    embeddings_storage_mode='gpu',
)

2023-11-10 09:41:21,154 ----------------------------------------------------------------------------------------------------
2023-11-10 09:41:21,156 Model: "RelationClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768,

100%|██████████| 391/391 [00:33<00:00, 11.63it/s]

2023-11-10 09:44:03,927 Evaluating as a multi-label problem: False


2023-11-10 09:44:03,966 DEV : loss 0.20799756050109863 - f1-score (macro avg)  0.3032
2023-11-10 09:44:04,435 ----------------------------------------------------------------------------------------------------
2023-11-10 09:44:20,205 epoch 2 - iter 102/1024 - loss 0.27005642 - time (sec): 15.77 - samples/sec: 51.75 - lr: 0.000022
2023-11-10 09:44:35,870 epoch 2 - iter 204/1024 - loss 0.26886919 - time (sec): 31.43 - samples/sec: 51.92 - lr: 0.000024
2023-11-10 09:44:51,468 epoch 2 - iter 306/1024 - loss 0.24865627 - time (sec): 47.03 - samples/sec: 52.05 - lr: 0.000026
2023-11-10 09:45:07,257 epoch 2 - iter 408/1024 - loss 0.24875280 - time (sec): 62.82 - samples/sec: 51.96 - lr: 0.000028
2023-11-10 09:45:23,194 epoch 2 - iter 510/1024 - loss 0.22617923 - time (sec): 78.76 - samples/sec: 51.80 - lr: 0.000030
2023-11-10 09:45:39,162 epoch 2 - iter 612/1024 - loss 0.21229374 - time (sec): 94.73 - samples/sec: 51.69 - lr: 0.000032
2023-11-10 09:45:55,007 epoch 2 - iter 714/1024 - loss 0.

100%|██████████| 391/391 [00:32<00:00, 12.15it/s]


...

2023-11-10 10:44:35,337 Evaluating as a multi-label problem: False
2023-11-10 10:44:35,394 DEV : loss 0.2806123197078705 - f1-score (macro avg)  0.7907


2023-11-10 10:44:36,642 ----------------------------------------------------------------------------------------------------
2023-11-10 10:44:36,648 Testing using last state of model ...


100%|██████████| 621/621 [00:51<00:00, 12.08it/s]


2023-11-10 10:45:28,134 Evaluating as a multi-label problem: False
2023-11-10 10:45:28,214 0.9833	0.9833	0.9833	0.9833
2023-11-10 10:45:28,219 
Results:
- F-score (micro) 0.9833
- F-score (macro) 0.8519
- Accuracy 0.9833

By class:
                         precision    recall  f1-score   support

                            0.9886    0.9941    0.9913      4722
 received_nobelprize_in     0.8542    0.7193    0.7810        57
                has_won     0.8750    0.7500    0.8077        56
        has_nationality     0.8537    0.8537    0.8537        41
received_nobelprize_for     0.8667    0.6047    0.7123        43
                born_on     0.9062    0.9355    0.9206        31
                died_on     0.8125    1.0000    0.8966        13

               accuracy                         0.9833      4963
              macro avg     0.8796    0.8367    0.8519      4963
           weighted avg     0.9827    0.9833    0.9826      4963

2023-11-10 10:45:28,224 --------------------------

{'test_score': 0.8518799591731039,
 'dev_score_history': [0.30324872735760405,
  0.45767416520019394,
  0.7109586912482148,
  0.8183984314215603,
  0.8264995101603897,
  0.8455084570850433,
  0.7940780009334762,
  0.7826235600300739,
  0.6977273540274987,
  0.7705078396868459,
  0.8061085802819999,
  0.7811996273678782,
  0.8376924117123954,
  0.7540133777475991,
  0.8256244926173119,
  0.8117683529772554,
  0.7587597642737218,
  0.7928211799483561,
  0.7907306782509602,
  0.7907306782509602],
 'train_loss_history': [0.4421313327071543,
  0.19548283539870565,
  0.1186104439416846,
  0.0723404538678962,
  0.05704137880469936,
  0.040246722279616405,
  0.026219445213345353,
  0.015770059697864394,
  0.022194670024294073,
  0.0201640980191223,
  0.007451911496737029,
  0.006263592299737058,
  0.004708399535028456,
  0.007113202604945628,
  0.0015880570519373899,
  0.0035276494785739924,
  8.878384218941862e-05,
  0.0006899052556953796,
  7.608930745421715e-07,
  3.493261327327111e-07],
 '

### Using 2-step
Here we load the trained models and see if they work as intended.

In [4]:
loaded_ner = SequenceTagger.load("data/comp1/ner/final-model.pt")
loaded_re: RelationClassifier = RelationClassifier.load('data/comp1/relations/final-model.pt')

2023-11-10 12:35:09,666 SequenceTagger predicts: Dictionary with 11 tags: O, B-Winner, I-Winner, B-Date, I-Date, B-Nationality, I-Nationality, B-Prizetype, I-Prizetype, B-Reason, I-Reason


In [4]:
sentence = Sentence('Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer. ')
loaded_ner.predict(sentence)
loaded_re.predict(sentence)
print(sentence.to_dict())
print(sentence.get_labels('relation'))

{'text': 'Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer.', 'all labels': [{'value': 'Winner', 'confidence': 0.9999329149723053}, {'value': 'Date', 'confidence': 0.9999633431434631}, {'value': 'Nationality', 'confidence': 0.9999721050262451}, {'value': 'Nationality', 'confidence': 0.9999365409215292}, {'value': 'Winner', 'confidence': 0.9998683929443359}, {'value': 'Winner', 'confidence': 0.9999294281005859}, {'value': 'Nationality', 'confidence': 0.9999485015869141}, {'value': 'Nationality', 'confidence': 0.5977905988693237}, {'value': 'Nationality', 'confidence': 0.999382495880127}, {'value': 'born_on', 'confidence': 0

Here we apply the 2-step Relation Extraction to our test set. We simply load the json created in label-studio, run both models sequentially and create two files. One file contains only the from text, to text and label information. The other contains both the entities, relations, their confidence and all tokens.

The latter file can be used for pruning low-confidence predictions.

In [ ]:
test_set = srsly.read_json('comp1_test_set.json')
all_relations = {}
document_dicts = []
for document in test_set:
  sentence = Sentence(document["data"]["text"])
  loaded_ner.predict(sentence)
  loaded_re.predict(sentence)
  sent_dict = sentence.to_dict()
  all_relations[document["data"]["title"]] = [(relation["from_text"], relation["to_text"], relation["labels"][0]["value"]) for relation in sent_dict["relations"] if len(relation["labels"][0]["value"]) > 0]
  document_dicts.append(sentence.to_dict())


srsly.write_json('comp1_relations.json', all_relations)
srsly.write_json('comp1_output.json', document_dicts)


# Component 2
For component 2 a very similar model is used.
The conll files and corpus are created in the same way as before.

In [ ]:
train_json = srsly.read_json('comp2_train_set.json')
make_conll(train_json, key_field='Abstract_new', file_name='data/comp2_train_set')

test_json = srsly.read_json('comp2_test_set.json')
make_conll(test_json, key_field='Abstract_new', file_name='data/comp2_test_set')

# Split the train file into train/dev split
with open('data/comp2_train_set.conll') as f:
  data = f.read()
  splits = data.split(2*os.linesep)
  splits = sorted(splits, key=lambda x: random.random())

  train = splits[:90]
  dev = splits[90:]

  write_conll('data/comp2/train.conll', train)
  write_conll('data/comp2/dev.conll', dev)

with open('data/comp2_test_set.conll') as f:
  data = f.read()
  splits = data.split(2*os.linesep)

  write_conll('data/comp2/test.conll', splits)


In [ ]:
columns = {1: 'text', 2: 'ner'}
data_folder = ''

corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='data/comp2/train.conll',
                              test_file='data/comp2/test.conll',
                              dev_file='data/comp2/dev.conll')
print(f'Corpus size: {len(corpus.train)}')

Note here the change in subtoken pooling, to include the mean of all subtokens, rather than only the first. This improved performance substantially for component 2.

In [ ]:
# tag to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

embeddings = TransformerWordEmbeddings(
    model='distilbert-base-cased',
    layers="-1",
    subtoken_pooling='mean',
    fine_tune=True,
    use_context=True,
    respect_document_boundaries=False,
)

tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,
        embeddings=embeddings,
        tag_dictionary=tag_dictionary,
        tag_type=tag_type,
        use_crf=False,
        use_rnn=False,
        reproject_embeddings=False,
        tag_format='BIO'
    )

In [ ]:
trainer : ModelTrainer = ModelTrainer(tagger, corpus)

trainer.fine_tune(
        'data/comp2/ner/',
        learning_rate=5e-05,
        mini_batch_size=16,
        max_epochs=100,
        embeddings_storage_mode='gpu', # cpu, gpu
        weight_decay=0.0,
    )

### RE component 2

In [ ]:
relation_label_dict = corpus.make_label_dictionary(label_type="relation")
relation_dictionary = corpus.make_label_dictionary("relation")
doc_embeddings = TransformerDocumentEmbeddings(model="distilbert-base-uncased", layers="-1", fine_tune=True)
relation_model: RelationClassifier = RelationClassifier(
    embeddings =doc_embeddings,
    label_dictionary = relation_dictionary,
    label_type="relation",
    entity_label_types="ner",
    entity_pair_labels={  # Define valid entity pair combinations, used as relation candidates
        ("Person", "Component"),
        ("Person", "Time"),
        ("Person", "Date"),
        ("Person", "Person"),
        ("Component", "Person"),
        ("Component", "Date"),
        ("Component", "Time"),
        ("Component", "Component")
    },
    zero_tag_value='',
    allow_unk_tag=False,
)

In [ ]:
trainer: ModelTrainer = ModelTrainer(model=relation_model, corpus=relation_model.transform_corpus(corpus))
trainer.fine_tune(
    'data/comp2/relations/',
    max_epochs=20,
    learning_rate=4e-5,
    mini_batch_size=8,
    main_evaluation_metric=("macro avg", "f1-score"),
    shuffle=True,
    embeddings_storage_mode='gpu',
)

### Using 2-step RE on the validation set
The models are used on the test set for component 2, the same as for component 1.


In [2]:
# Load both models, identical to comp 1
loaded_ner = SequenceTagger.load("data/comp2/ner/final-model.pt")
loaded_re: RelationClassifier = RelationClassifier.load('data/comp2/relations/final-model.pt')

2023-11-10 12:33:24,361 SequenceTagger predicts: Dictionary with 9 tags: O, B-Component, I-Component, B-Person, I-Person, B-Time, I-Time, B-Date, I-Date


In [ ]:
# process all test data and store the result
test_set = srsly.read_json('comp2_test_set.json')
all_relations = {}
document_dicts = []
for document in test_set:
  sentence = Sentence(document["data"]["Abstract_new"])
  loaded_ner.predict(sentence)
  loaded_re.predict(sentence)
  sent_dict = sentence.to_dict()
  unique_id = " ".join([str(document["data"]["EventDt"]), document["data"]["IPFCode"], document["data"]["IPFShortDesc"], str(document["inner_id"])])
  all_relations[unique_id] = [(relation["from_text"], relation["to_text"], relation["labels"][0]["value"]) for relation in sent_dict["relations"] if len(relation["labels"][0]["value"]) > 0]
  document_dicts.append(sentence.to_dict())


srsly.write_json('comp2_relations.json', all_relations)
srsly.write_json('comp2_output.json', document_dicts)
